In [ ]:
# 패키지 임포트

import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from torchvision import datasets, transforms
from tqdm.notebook import tqdm

In [ ]:
# mnist 데이터 다운로드

mnist_train = datasets.MNIST(root="./datasets", train=True, transform=transforms.ToTensor(), download=True)
mnist_test = datasets.MNIST(root="./datasets", train=False, transform=transforms.ToTensor(), download=True)
train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=100, shuffle=False)

In [ ]:
# 네트워크 정의

input_size = 784
hidden_sizes = [128, 64]
output_size = 10

# nn.Sequential을 통해 입력부터 출력까지 순차적으로 네트워크 구조를 정의
# nn.Linear는 선형 곱 레이어
# nn.ReLU는 ReLU 활성화 레이어
# nn.LogSoftmax는 Softmax 활성화 레이어어
model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], output_size),
                      nn.LogSoftmax(dim=1))

In [ ]:
# 손실함수 정의
criterion = nn.CrossEntropyLoss()

# 최적화 알고리즘 정의
# SGD는 Stochastic Gradient Descent를 의미미
optimizer = torch.optim.SGD(model.parameters(), lr=0.09)

In [ ]:
# 훈련
epochs = 15
for e in range(epochs):
    running_loss = 0
    for images, labels in train_loader:
        images = images.view(images.shape[0], -1)

        #기울기 값값 초기화
        optimizer.zero_grad()
        
        #모델 계산
        output = model(images)

        #손실 계산
        loss = criterion(output, labels)
        
        #역전파
        loss.backward()
        
        #최적화
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print("Epoch {} - Training loss: {}".format(e, running_loss/len(train_loader)))


In [ ]:
# 테스트
correct = 0
total = len(mnist_test)
with torch.no_grad():
    # Iterate through test set minibatchs 
    for images, labels in tqdm(test_loader):
        # Forward pass
        #x = images.view(-1, 28*28)
        x = images.view(images.shape[0], -1)
        y = model(x)
        
        predictions = torch.argmax(y, dim=1)
        correct += torch.sum((predictions == labels).float())
    
print('Test accuracy: {}'.format(correct/total))
